In [1]:
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
import xgboost
import re
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFECV
from sklearn.ensemble import VotingClassifier

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import LeaveOneOut

Using TensorFlow backend.


In [2]:
combined_data=pickle.load( open(\
    "D:/Projects/Prognostic Predictions/Proccessed data/final_data.pickle", "rb" ) )


In [3]:
combined_data.to_csv("Final_data_set.csv",index=False)

In [12]:

def my_custom_loss_func(y_true, y_pred):
    threshold=0.98
    fpr=1
    print(y_pred)
    while fpr>0.009 and threshold<1:
        threshold+=0.01
        y_pred  = (y_pred  >= threshold).astype('int')
        if y_pred==y_true:
            return 0
        elif y_pred==1:
            return -10
        elif y_pred==0:
            return -1
    return 0

In [13]:
from tqdm import tqdm
def model_iterations_loocv(model,param_grid):
    ksm = SMOTE(random_state=42)
    y_pred_list=[]
    loo_hyper_parameter_selection = LeaveOneOut()
    loo_testing = LeaveOneOut()
    y_index=[]
    X=combined_data.drop(columns=["PID","outcome"]).values
    Y=combined_data["outcome"].values
    loo_testing.get_n_splits(X)
    mm=MinMaxScaler()
#      include hyper selection
    
    for train_index, test_index in tqdm(loo_testing.split(X)):
        
        X_train,X_test,Y_train,Y_test=X[train_index],X[test_index],Y[train_index],Y[test_index]
#         print(np.unique(Y_train))
            
        estimator=model
        random_search = RandomizedSearchCV(estimator, param_distributions=param_grid, n_iter=20, 
#                                            scoring='roc_auc',
                                           scoring=make_scorer(my_custom_loss_func,needs_proba=True),
                                           n_jobs=-1, cv=loo_hyper_parameter_selection.split(X_train,Y_train),
                                           random_state=42)
        random_search.fit(X_train, Y_train)
        best_model = random_search.best_estimator_
#         print(best_model)
        best_model.fit(X_train, Y_train)
#         X_train,Y_train=ksm.fit_resample(X_train, Y_train)
#         X_train=mm.fit_transform(X_train)
#         X_test=mm.fit_transform(X_test)
        
        y_pred=best_model.predict_proba(X_test)
        y_pred=(y_pred[:,1]>=0.99).astype('int')
        y_pred_list.append(y_pred[0])
        y_index.append(test_index[0])   
    return y_pred_list,y_index


In [16]:
model=XGBClassifier(random_state=42)

param_grid = {
#         'tree_method':['gpu_hist'],  #### include if you have a gpu compatible version of XGBOOST
        'reg_lambda':[0.5,0.8],
        'min_child_weight': [ 5, 10],
        'gamma': [0.1,0.5],
#         'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.2],
        'learning_rate':[0.3,0.5],
        'max_depth': [ 4,8],
        'random_state':[42],
        'scale_pos_weight':[0.8],"n_jobs":[-1]
        }



output=model_iterations_loocv(model,param_grid)
y_pred,y_test_index=output
Y_test=combined_data["outcome"].values
tn, fp, fn, tp=confusion_matrix(Y_test.flatten()[y_test_index], y_pred).ravel()
fpr=fp/(fp+tn)
tpr=tp/(tp+fn)
print("TPR:", tpr)
print("FPR:",fpr)
y_test_index=np.array(y_test_index)
y_test_index[Y_test.flatten()[y_test_index]!= y_pred]
Y_test.flatten()[Y_test.flatten()[y_test_index]!= y_pred]